In [4]:
import tkinter as tk
from tkinter import messagebox, ttk
import joblib
import numpy as np
from xgboost import XGBRegressor

# Load the XGBoost models
model_paths = {
    "Central Valley (Stn. 39) - RCP 4.5": 'C:\\Users\\asus1\\Desktop\\xgb1.joblib',
    "Central Valley (Stn. 39) - RCP 8.5": 'C:\\Users\\asus1\\Desktop\\xgb2.joblib',
    "Imperial Valley (Stn. 87) - RCP 4.5": 'C:\\Users\\asus1\\Desktop\\xgb3.joblib',
    "Imperial Valley (Stn. 87) - RCP 8.5": 'C:\\Users\\asus1\\Desktop\\xgb4.joblib',
}

models = {}
for key, path in model_paths.items():
    try:
        models[key] = joblib.load(path)
    except FileNotFoundError:
        models[key] = XGBRegressor()  # Ensure the models are trained beforehand

# Labels for input fields
labels = {
    'X1': r'Tmax (℃)',
    'X2': r'Tmin (℃)', 
    'X3': r'U (m/s)',  
    'X4': r'RH (%)', 
}

# Define the GUI application
class PredictionApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Modeling Climate Change Effect on Reference Evapotranspiration")
        self.geometry("1000x600")
        self.configure(bg="#f4f4f9")  # Light grey background for a formal look

        # Title and developers
        title_label = tk.Label(self, text="Modeling Climate Change Effect on Reference Evapotranspiration in Semi-Arid Regions Using Explainable Hybrid Gradient Boosting Models", font=("CMU Concrete", 17, "bold"), wraplength=850, justify="center", bg="#f4f4f9")
        title_label.pack(pady=10)

        dev_label = tk.Label(self, text="Developers: Mohamed Kamel Elshaarawy, Romysaa Elasbah, Mohamed Elsayed Gabr,\nKhalid M. Bali & Mohamed Galal Eltarabily", font=("CMU Concrete", 13), wraplength=850, justify="center", bg="#f4f4f9")
        dev_label.pack(pady=5)

        # Model and Zone Selection
        selection_frame = tk.Frame(self, bg="#f4f4f9")
        selection_frame.pack(pady=10)

        # Zone selection dropdown
        tk.Label(selection_frame, text="Select Zone: ", font=("CMU Concrete", 13, "bold"), bg="#f4f4f9").grid(row=0, column=0, padx=10)
        self.zone_var = tk.StringVar()
        self.zone_menu = ttk.Combobox(selection_frame, textvariable=self.zone_var, font=("CMU Concrete", 13), values=["Central Valley (Stn. 39)", "Imperial Valley (Stn. 87)"], state="readonly")
        self.zone_menu.grid(row=0, column=1, padx=10)

        # RCP selection dropdown
        tk.Label(selection_frame, text="Select RCP Model: ", font=("CMU Concrete", 13, "bold"), bg="#f4f4f9").grid(row=1, column=0, padx=10)
        self.rcp_var = tk.StringVar()
        self.rcp_menu = ttk.Combobox(selection_frame, textvariable=self.rcp_var, font=("CMU Concrete", 13), values=["RCP 4.5", "RCP 8.5"], state="readonly")
        self.rcp_menu.grid(row=1, column=1, padx=10)

        # Create a frame to hold both the input and prediction panels side by side
        main_frame = tk.Frame(self, bg="#f4f4f9")
        main_frame.pack(pady=10, padx=10, fill="both", expand=True)

        # Input fields for X1, X2, X3, and X4 inside a LabelFrame with a border
        input_frame = tk.LabelFrame(main_frame, text="Input Parameters", padx=15, pady=10, font=("CMU Concrete", 13, "bold"), bg="#ffffff", bd=1)
        input_frame.grid(row=0, column=0, padx=10, pady=5, sticky="n")

        self.entries = {}
        row = 0
        for key, label_text in labels.items():
            tk.Label(input_frame, text=label_text, font=("CMU Concrete", 13), bg="#ffffff").grid(row=row, column=0, padx=5, pady=5, sticky="w")
            entry = tk.Entry(input_frame, font=("CMU Concrete", 13, "bold"))
            entry.grid(row=row, column=1, padx=5, pady=5)
            self.entries[key] = entry
            row += 1

        # Prediction Section inside a LabelFrame with a border (placed beside input frame)
        prediction_frame = tk.LabelFrame(main_frame, text="Prediction Output", padx=15, pady=10, font=("CMU Concrete", 13, "bold"), bg="#ffffff", bd=1)
        prediction_frame.grid(row=0, column=1, padx=10, pady=5, sticky="n")

        self.prediction_label = tk.Label(prediction_frame, text="ETo (mm): Reference Evapotranspiration = ", font=("CMU Concrete", 13), bg="#ffffff")
        self.prediction_label.grid(row=0, column=0, padx=5, pady=5)

        self.prediction_result = tk.Label(prediction_frame, text="", font=("CMU Concrete", 13, "bold"), bg="#ffffff")
        self.prediction_result.grid(row=0, column=1, padx=5, pady=5)

        # Predict and Clear buttons placed below the two panels
        button_frame = tk.Frame(self, bg="#f4f4f9")
        button_frame.pack(pady=15)

        predict_button = tk.Button(button_frame, text="Predict", command=self.predict, font=("CMU Concrete", 13, "bold"), bg="#b0bec5", padx=20)
        predict_button.grid(row=0, column=0, padx=15, pady=10)

        clear_button = tk.Button(button_frame, text="Clear", command=self.clear, font=("CMU Concrete", 13, "bold"), bg="#b0bec5", padx=20)
        clear_button.grid(row=0, column=1, padx=15, pady=10)

        # Make GUI responsive
        self.grid_rowconfigure(0, weight=1)
        self.grid_columnconfigure(0, weight=1)

    def predict(self):
        try:
            # Get zone and RCP model selected
            zone = self.zone_var.get()
            rcp = self.rcp_var.get()

            # Ensure both selections are made
            if not zone or not rcp:
                raise ValueError("Please select both a Zone and an RCP model.")

            # Determine which model to use
            model_key = f"{zone} - {rcp}"
            model = models.get(model_key)

            if model is None:
                raise ValueError("Model not available for the selected Zone and RCP model.")

            # Collect inputs for X1, X2, X3, and X4
            inputs = []
            for key in labels:
                value = self.entries[key].get().strip()
                if value == "":
                    raise ValueError(f"Please enter a valid number for {key}.")
                try:
                    inputs.append(float(value))  # Convert input to float
                except ValueError:
                    raise ValueError(f"Invalid input for {key}: {value} is not a number.")

            # Convert inputs to the correct format for prediction
            inputs = np.array(inputs).reshape(1, -1)
            
            # Predict the output using the selected model
            prediction = model.predict(inputs)
            
            # Display the predicted value (ETo in mm/day)
            self.prediction_result.config(text=f"{prediction[0]:.2f} mm/day")
        
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def clear(self):
        # Clear all input fields and prediction result
        for entry in self.entries.values():
            entry.delete(0, tk.END)
        self.prediction_result.config(text="")

# Run the application
if __name__ == "__main__":
    app = PredictionApp()
    app.mainloop()


In [ ]:
import tkinter as tk
from tkinter import messagebox, ttk
import joblib
import numpy as np
from xgboost import XGBRegressor

# Load the XGBoost models
model_paths = {
    "Central Valley (Stn. 39) - RCP 4.5": 'C:\\Users\\asus1\\Desktop\\xgb1.joblib',
    "Central Valley (Stn. 39) - RCP 8.5": 'C:\\Users\\asus1\\Desktop\\xgb2.joblib',
    "Imperial Valley (Stn. 87) - RCP 4.5": 'C:\\Users\\asus1\\Desktop\\xgb3.joblib',
    "Imperial Valley (Stn. 87) - RCP 8.5": 'C:\\Users\\asus1\\Desktop\\xgb4.joblib',
}

models = {}
for key, path in model_paths.items():
    try:
        models[key] = joblib.load(path)
    except FileNotFoundError:
        models[key] = XGBRegressor()  # Ensure the models are trained beforehand

# Labels for input fields
labels = {
    'X1': r'Tmax (℃)',
    'X2': r'Tmin (℃)', 
    'X3': r'U (m/s)',  
    'X4': r'RH (%)', 
}

# Define the GUI application
class PredictionApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Modeling Climate Change Effect on Reference Evapotranspiration")
        self.geometry("1000x600")
        self.configure(bg="#f4f4f9")  # Light grey background for a formal look

        # Title and developers
        title_label = tk.Label(self, text="Modeling Climate Change Effect on Reference Evapotranspiration in Semi-Arid Regions Using Explainable Hybrid Gradient Boosting Models", font=("CMU Concrete", 17, "bold"), wraplength=850, justify="center", bg="#f4f4f9")
        title_label.pack(pady=10)

        dev_label = tk.Label(self, text="Developers: Mohamed Kamel Elshaarawy, Romysaa Elasbah, Mohamed Elsayed Gabr,\nKhalid M. Bali & Mohamed Galal Eltarabily", font=("CMU Concrete", 13), wraplength=850, justify="center", bg="#f4f4f9")
        dev_label.pack(pady=5)

        # Model and Zone Selection
        selection_frame = tk.Frame(self, bg="#f4f4f9")
        selection_frame.pack(pady=10)

        # Zone selection dropdown with increased width
        tk.Label(selection_frame, text="Select Zone: ", font=("CMU Concrete", 13, "bold"), bg="#f4f4f9").grid(row=0, column=0, padx=10)
        self.zone_var = tk.StringVar()
        self.zone_menu = ttk.Combobox(selection_frame, textvariable=self.zone_var, font=("CMU Concrete", 13), values=["Central Valley (Stn. 39)", "Imperial Valley (Stn. 87)"], state="readonly", width=30)
        self.zone_menu.grid(row=0, column=1, padx=10)

        # RCP selection dropdown with increased width
        tk.Label(selection_frame, text="Select RCP Model: ", font=("CMU Concrete", 13, "bold"), bg="#f4f4f9").grid(row=1, column=0, padx=10)
        self.rcp_var = tk.StringVar()
        self.rcp_menu = ttk.Combobox(selection_frame, textvariable=self.rcp_var, font=("CMU Concrete", 13), values=["RCP 4.5", "RCP 8.5"], state="readonly", width=30)
        self.rcp_menu.grid(row=1, column=1, padx=10)

        # Create a frame to hold both the input and prediction panels side by side
        main_frame = tk.Frame(self, bg="#f4f4f9")
        main_frame.pack(pady=10, padx=10, fill="both", expand=True)

        # Input fields for X1, X2, X3, and X4 inside a LabelFrame with a border
        input_frame = tk.LabelFrame(main_frame, text="Input Parameters", padx=15, pady=10, font=("CMU Concrete", 13, "bold"), bg="#ffffff", bd=1)
        input_frame.grid(row=0, column=0, padx=10, pady=5, sticky="n")

        self.entries = {}
        row = 0
        for key, label_text in labels.items():
            tk.Label(input_frame, text=label_text, font=("CMU Concrete", 13), bg="#ffffff").grid(row=row, column=0, padx=5, pady=5, sticky="w")
            entry = tk.Entry(input_frame, font=("CMU Concrete", 13, "bold"))
            entry.grid(row=row, column=1, padx=5, pady=5)
            self.entries[key] = entry
            row += 1

        # Prediction Section inside a LabelFrame with a border (placed beside input frame)
        prediction_frame = tk.LabelFrame(main_frame, text="Prediction Output", padx=15, pady=10, font=("CMU Concrete", 13, "bold"), bg="#ffffff", bd=1)
        prediction_frame.grid(row=0, column=1, padx=10, pady=5, sticky="n")

        self.prediction_label = tk.Label(prediction_frame, text="ETo (mm): Reference Evapotranspiration = ", font=("CMU Concrete", 13), bg="#ffffff")
        self.prediction_label.grid(row=0, column=0, padx=5, pady=5)

        self.prediction_result = tk.Label(prediction_frame, text="", font=("CMU Concrete", 13, "bold"), bg="#ffffff")
        self.prediction_result.grid(row=0, column=1, padx=5, pady=5)

        # Predict and Clear buttons placed below the two panels
        button_frame = tk.Frame(self, bg="#f4f4f9")
        button_frame.pack(pady=15)

        predict_button = tk.Button(button_frame, text="Predict", command=self.predict, font=("CMU Concrete", 13, "bold"), bg="#b0bec5", padx=20)
        predict_button.grid(row=0, column=0, padx=15, pady=10)

        clear_button = tk.Button(button_frame, text="Clear", command=self.clear, font=("CMU Concrete", 13, "bold"), bg="#b0bec5", padx=20)
        clear_button.grid(row=0, column=1, padx=15, pady=10)

        # Make GUI responsive
        self.grid_rowconfigure(0, weight=1)
        self.grid_columnconfigure(0, weight=1)

    def predict(self):
        try:
            # Get zone and RCP model selected
            zone = self.zone_var.get()
            rcp = self.rcp_var.get()

            # Ensure both selections are made
            if not zone or not rcp:
                raise ValueError("Please select both a Zone and an RCP model.")

            # Determine which model to use
            model_key = f"{zone} - {rcp}"
            model = models.get(model_key)

            if model is None:
                raise ValueError("Model not available for the selected Zone and RCP model.")

            # Collect inputs for X1, X2, X3, and X4
            inputs = []
            for key in labels:
                value = self.entries[key].get().strip()
                if value == "":
                    raise ValueError(f"Please enter a valid number for {key}.")
                try:
                    inputs.append(float(value))  # Convert input to float
                except ValueError:
                    raise ValueError(f"Invalid input for {key}: {value} is not a number.")

            # Convert inputs to the correct format for prediction
            inputs = np.array(inputs).reshape(1, -1)
            
            # Predict the output using the selected model
            prediction = model.predict(inputs)
            
            # Display the predicted value (ETo in mm/day)
            self.prediction_result.config(text=f"{prediction[0]:.2f} mm/day")
        
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def clear(self):
        # Clear all input fields and prediction result
        for entry in self.entries.values():
            entry.delete(0, tk.END)
        self.prediction_result.config(text="")

# Run the application
if __name__ == "__main__":
    app = PredictionApp()
    app.mainloop()
